In [22]:
from ultralytics import YOLO
import cv2
import math
from gtts import gTTS
import os
from collections import Counter
model = YOLO('yolov8m_saved_model/yolov8m_float32.tflite')

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [23]:
def audio(path):
    
    language = 'en'
    results = model(path)
    classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
                  "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
                  "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
                  "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
                  "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
                  "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
                  "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
                  "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
                  "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
                  "teddy bear", "hair drier", "t|oothbrush"
                  ]

    x = "There "  # Initialize x
    for result in results:
        boxes = result.boxes.cls
        boxes_int = [int(x) for x in boxes]
        counter = Counter(boxes_int)

        print(counter)  # Check what boxes contains
        for i, (box, count) in enumerate(counter.items()):
            if i != len(counter) - 1 and i != 0:
                if count == 1:
                    x += f"a {classNames[int(box)]} , "
                else:
                    x += f"{count} {classNames[int(box)]}s , "
            elif i == 0:
                if count == 1:
                    x += f"is a {classNames[int(box)]} , "
                else:
                    x += f"are {count} {classNames[int(box)]}s , "
            else:
                if count == 1:
                    x += f"and a {classNames[int(box)]} . "
                else:
                    x += f"and {count} {classNames[int(box)]}s . "

    print(x)
    tts = gTTS(text=x, lang=language, slow=False)

    tts.save("output.mp3")
    os.system("start output.mp3")
    

In [24]:
def detect():
    cap = cv2.VideoCapture(0)
    cap.set(3, 1920)
    cap.set(4, 1080)
    
    # Main loop
    while True:
        success, img = cap.read()
        cv2.imshow('Webcam', img)
        
        key = cv2.waitKey(1)
        
        # Call the audio function when 'f' is pressed
        if key == ord('r'):
            cv2.imwrite("temp_img.jpg", img)  # Save a temporary image
            audio("temp_img.jpg")  # Call the audio function with the path to the temporary image
        
        elif key == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [29]:
import cv2
import pyttsx3
def motion():
    # Video capture initialization
    cap = cv2.VideoCapture(0)
    cap.set(3, 1920)
    cap.set(4, 1080)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1920, 1080))

    # Initialize background subtractor
    backSub = cv2.createBackgroundSubtractorMOG2()

    # Counter to record frames
    frame_count = 0

    # Flag to indicate if moving objects are detected
    moving_objects = False

    # Main loop
    while True:
        success, img = cap.read()

        if not success:
            break

        # Apply background subtraction
        fgMask = backSub.apply(img)

        # Get motion detection bounding box
        contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 500:  # Adjust this threshold as needed
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                moving_objects = True  # Set the flag if any significant motion is detected

        # Write the frame to the output video
        out.write(img)

        # Increment frame counter
        frame_count += 1

        # Stop recording after 100 frames (5 seconds)
        if frame_count == 50:
            break

        # Print whether there is movement detected
        if moving_objects:
            X="Moving objects detected"
        else:
            X="No moving objects detected"

        cv2.imshow('Motion Detection', img)
        cv2.waitKey(1)

    # Release video capture and writer objects
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    pyttsx3.speak(X)

# Test the motion function
motion()
detect()


image 1/1 C:\Users\User\temp_img.jpg: 640x640 2 persons, 954.5ms
Speed: 3.5ms preprocess, 954.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Counter({0: 2})
There are 2 persons , 

image 1/1 C:\Users\User\temp_img.jpg: 640x640 2 persons, 972.0ms
Speed: 3.0ms preprocess, 972.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Counter({0: 2})
There are 2 persons , 

image 1/1 C:\Users\User\temp_img.jpg: 640x640 2 persons, 1 bed, 972.1ms
Speed: 4.0ms preprocess, 972.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Counter({0: 2, 59: 1})
There are 2 persons , and a bed . 

image 1/1 C:\Users\User\temp_img.jpg: 640x640 2 persons, 1 dog, 941.1ms
Speed: 4.0ms preprocess, 941.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Counter({0: 2, 16: 1})
There are 2 persons , and a dog . 

image 1/1 C:\Users\User\temp_img.jpg: 640x640 5 persons, 1 bottle, 1 cell phone, 964.3ms
Speed: 8.0ms preprocess, 964.3ms inference, 1.